<a href="https://colab.research.google.com/github/HRI328/Supervised_ML/blob/main/Supervised_Machine_Learning_Model_Builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Supervised Machine Learning Model Builder**

# With Flexible Feature Selection: Chi2 (categorical) + PCA (numerical)

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, chi2
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, mean_squared_error, mean_absolute_error, r2_score
)
from scipy.stats import chi2_contingency

# Classification models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Regression models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

# Boosting models
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("⚠️  XGBoost not installed. Run: pip install xgboost")

try:
    import catboost as cb
    CATBOOST_AVAILABLE = True
except ImportError:
    CATBOOST_AVAILABLE = False
    print("⚠️  CatBoost not installed. Run: pip install catboost")

import warnings
warnings.filterwarnings('ignore')



⚠️  CatBoost not installed. Run: pip install catboost


**Machine Learning Pipeline**

In [2]:

class MLModelBuilder:
    def __init__(self, task='classification', random_state=42):
        self.task = task
        self.random_state = random_state
        self.models = {}
        self.results = {}
        self.best_model = None
        self.scaler = StandardScaler()

        # Preprocessing state
        self.categorical_features = []
        self.numerical_features = []
        self.categorical_encoders = {}
        self.feature_names = []

        # Feature selection state
        # Tracks what user chose: 'all', 'chi2', 'pca', or 'chi2+pca'
        self.feature_mode = None
        self.pca = None
        self.chi2_selector = None
        self.chi2_k = None

        # Index tracking after encoding
        self.num_indices = []
        self.cat_indices = []

        # Define models
        if task == 'classification':
            self.models = {
                'Logistic Regression': LogisticRegression(random_state=random_state, max_iter=1000),
                'Decision Tree': DecisionTreeClassifier(random_state=random_state),
                'Random Forest': RandomForestClassifier(random_state=random_state, n_jobs=-1),
                'Gradient Boosting': GradientBoostingClassifier(random_state=random_state),
                'SVM': SVC(random_state=random_state, probability=True),
                'KNN': KNeighborsClassifier()
            }
            if XGBOOST_AVAILABLE:
                self.models['XGBoost'] = xgb.XGBClassifier(
                    random_state=random_state, n_jobs=-1,
                    eval_metric='logloss', use_label_encoder=False
                )
            if CATBOOST_AVAILABLE:
                self.models['CatBoost'] = cb.CatBoostClassifier(
                    random_state=random_state, verbose=0, thread_count=-1
                )
        else:
            self.models = {
                'Linear Regression': LinearRegression(),
                'Ridge': Ridge(random_state=random_state),
                'Lasso': Lasso(random_state=random_state),
                'Decision Tree': DecisionTreeRegressor(random_state=random_state),
                'Random Forest': RandomForestRegressor(random_state=random_state, n_jobs=-1),
                'Gradient Boosting': GradientBoostingRegressor(random_state=random_state),
                'SVR': SVR()
            }
            if XGBOOST_AVAILABLE:
                self.models['XGBoost'] = xgb.XGBRegressor(random_state=random_state, n_jobs=-1)
            if CATBOOST_AVAILABLE:
                self.models['CatBoost'] = cb.CatBoostRegressor(
                    random_state=random_state, verbose=0, thread_count=-1
                )

    # =========================================================================
    # STEP 1: PREPARE DATA & ENCODE CATEGORICAL
    # =========================================================================

    def prepare_data(self, X, y, test_size=0.2):
        """
        Split data and encode categorical features.
        Does NOT scale yet — scaling happens after feature selection.
        """
        print("=" * 70)
        print("DATA PREPARATION")
        print("=" * 70)

        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])

        self.feature_names = X.columns.tolist()
        self.categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
        self.numerical_features = X.select_dtypes(include=[np.number]).columns.tolist()

        print(f"✓ Numerical features ({len(self.numerical_features)}): {self.numerical_features}")
        print(f"✓ Categorical features ({len(self.categorical_features)}): {self.categorical_features}")

        # Split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=self.random_state
        )
        print(f"✓ Train: {X_train.shape[0]} | Test: {X_test.shape[0]}")

        # Encode categorical
        if len(self.categorical_features) > 0:
            X_train, X_test = self._encode(X_train, X_test)

        # Convert to numpy
        self.X_train = X_train.values.astype(float)
        self.X_test = X_test.values.astype(float)
        self.y_train = y_train
        self.y_test = y_test

        # Track indices after encoding
        all_cols = list(X_train.columns)
        self.num_indices = [i for i, c in enumerate(all_cols) if c in self.numerical_features]
        self.cat_indices = [i for i, c in enumerate(all_cols) if c not in self.numerical_features]
        self.encoded_col_names = all_cols

        print(f"✓ Numerical indices: {self.num_indices}")
        print(f"✓ Categorical indices: {self.cat_indices}")
        print(f"✓ Total features after encoding: {self.X_train.shape[1]}")

        return self.X_train, self.X_test, self.y_train, self.y_test

    def _encode(self, X_train, X_test):
        """Encode categorical features using one-hot or label encoding."""
        print("\n  Encoding categorical features...")
        X_train_enc = X_train.copy()
        X_test_enc = X_test.copy()

        for col in self.categorical_features:
            n_unique = X_train[col].nunique()
            if n_unique <= 10:
                # One-hot encode
                enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
                train_encoded = enc.fit_transform(X_train[[col]])
                test_encoded = enc.transform(X_test[[col]])
                new_cols = [f"{col}_{cat}" for cat in enc.categories_[0]]

                train_df = pd.DataFrame(train_encoded, columns=new_cols, index=X_train_enc.index)
                test_df = pd.DataFrame(test_encoded, columns=new_cols, index=X_test_enc.index)

                X_train_enc = pd.concat([X_train_enc.drop(col, axis=1), train_df], axis=1)
                X_test_enc = pd.concat([X_test_enc.drop(col, axis=1), test_df], axis=1)
                self.categorical_encoders[col] = enc
                print(f"    • {col}: one-hot → {len(new_cols)} columns")
            else:
                # Label encode
                enc = LabelEncoder()
                X_train_enc[col] = enc.fit_transform(X_train[col].astype(str))
                X_test_enc[col] = enc.transform(X_test[col].astype(str))
                self.categorical_encoders[col] = enc
                print(f"    • {col}: label encoded ({n_unique} unique)")

        return X_train_enc, X_test_enc

    # =========================================================================
    # STEP 2: FLEXIBLE FEATURE SELECTION
    # User picks one of 4 modes:
    #   Mode A: all numerical + all categorical (no selection)
    #   Mode B: all numerical + chi2 selected categorical
    #   Mode C: PCA numerical + all categorical
    #   Mode D: PCA numerical + chi2 selected categorical
    # =========================================================================

    def select_features(self, mode='A', pca_variance=0.95, pca_n_components=None, chi2_k=5):
        """
        Flexible feature selection. Pick a mode:

        Args:
            mode: One of:
                'A' — All numerical + All categorical
                'B' — All numerical + Chi2 selected categorical
                'C' — PCA numerical + All categorical
                'D' — PCA numerical + Chi2 selected categorical
            pca_variance: Variance threshold for auto PCA (used if pca_n_components is None)
            pca_n_components: Fixed number of PCA components (overrides pca_variance)
            chi2_k: Number of categorical features to keep via Chi2
        """
        mode = mode.upper()
        assert mode in ['A', 'B', 'C', 'D'], "Mode must be 'A', 'B', 'C', or 'D'"
        self.feature_mode = mode
        self.chi2_k = chi2_k

        print("\n" + "=" * 70)
        print(f"FEATURE SELECTION — MODE {mode}")
        print("=" * 70)
        print(f"  A = All num + All cat")
        print(f"  B = All num + Chi2 cat")
        print(f"  C = PCA num + All cat")
        print(f"  D = PCA num + Chi2 cat")
        print(f"  → Selected: Mode {mode}")
        print()

        X_train_num = self.X_train[:, self.num_indices]
        X_test_num = self.X_test[:, self.num_indices]
        X_train_cat = self.X_train[:, self.cat_indices] if len(self.cat_indices) > 0 else None
        X_test_cat = self.X_test[:, self.cat_indices] if len(self.cat_indices) > 0 else None

        # --- Numerical part ---
        if mode in ['C', 'D']:
            X_train_num, X_test_num = self._apply_pca(X_train_num, X_test_num, pca_n_components, pca_variance)
        else:
            print(f"✓ Numerical: keeping all {X_train_num.shape[1]} features")

        # --- Categorical part ---
        if mode in ['B', 'D'] and X_train_cat is not None:
            X_train_cat, X_test_cat = self._apply_chi2(X_train_cat, X_test_cat, chi2_k)
        elif X_train_cat is not None:
            print(f"✓ Categorical: keeping all {X_train_cat.shape[1]} features")
        else:
            print("✓ Categorical: none detected")

        # --- Combine ---
        if X_train_cat is not None:
            self.X_train = np.hstack([X_train_num, X_train_cat])
            self.X_test = np.hstack([X_test_num, X_test_cat])
        else:
            self.X_train = X_train_num
            self.X_test = X_test_num

        # --- Scale after feature selection ---
        self.scaler.fit(self.X_train)
        self.X_train = self.scaler.transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)

        print(f"\n✓ Scaling applied")
        print(f"✓ Final shape — Train: {self.X_train.shape} | Test: {self.X_test.shape}")

        return self.X_train, self.X_test

    def _apply_pca(self, X_train_num, X_test_num, n_components, variance_threshold):
        """Internal: apply PCA on numerical features only."""
        print(f"✓ Numerical: applying PCA...")

        if n_components is None:
            # Auto-select based on variance
            pca_temp = PCA(random_state=self.random_state)
            pca_temp.fit(X_train_num)
            cumvar = np.cumsum(pca_temp.explained_variance_ratio_)
            n_components = int(np.argmax(cumvar >= variance_threshold) + 1)
            print(f"    Auto-selected {n_components} components ({variance_threshold*100}% variance)")
        else:
            n_components = min(n_components, X_train_num.shape[1])
            print(f"    Using {n_components} components (fixed)")

        self.pca = PCA(n_components=n_components, random_state=self.random_state)
        X_train_pca = self.pca.fit_transform(X_train_num)
        X_test_pca = self.pca.transform(X_test_num)

        total_var = self.pca.explained_variance_ratio_.sum()
        print(f"    {X_train_num.shape[1]} features → {n_components} PCA components (variance: {total_var*100:.2f}%)")

        return X_train_pca, X_test_pca

    def _apply_chi2(self, X_train_cat, X_test_cat, k):
        """Internal: apply Chi2 selection on categorical features only."""
        k = min(k, X_train_cat.shape[1])
        print(f"✓ Categorical: applying Chi2 (selecting {k} of {X_train_cat.shape[1]})...")

        # Chi2 requires non-negative values
        cat_min = X_train_cat.min()
        X_train_nn = X_train_cat - cat_min + 1e-10
        X_test_nn = X_test_cat - cat_min + 1e-10

        self.chi2_selector = SelectKBest(score_func=chi2, k=k)
        X_train_sel = self.chi2_selector.fit_transform(X_train_nn, self.y_train)
        X_test_sel = self.chi2_selector.transform(X_test_nn)

        # Store offset for prediction
        self.chi2_min_offset = cat_min

        # Print selected features
        selected_mask = self.chi2_selector.get_support()
        cat_col_names = [self.encoded_col_names[i] for i in self.cat_indices]
        selected_names = [name for name, sel in zip(cat_col_names, selected_mask) if sel]
        scores = self.chi2_selector.scores_

        print(f"    Selected categorical features:")
        for name, sc in zip(selected_names, scores[selected_mask]):
            print(f"      • {name}: chi2 = {sc:.4f}")

        return X_train_sel, X_test_sel

    def plot_pca_analysis(self, figsize=(15, 5)):
        """Visualize PCA results (only if PCA was used)."""
        if self.pca is None:
            print("✗ PCA not applied. Use mode 'C' or 'D' in select_features().")
            return

        fig, axes = plt.subplots(1, 3, figsize=figsize)
        ev = self.pca.explained_variance_ratio_

        # 1. Per-component variance
        axes[0].bar(range(1, len(ev) + 1), ev, color='steelblue', alpha=0.7)
        axes[0].set_xlabel('Principal Component')
        axes[0].set_ylabel('Variance Ratio')
        axes[0].set_title('Variance by Component')
        axes[0].grid(axis='y', alpha=0.3)

        # 2. Cumulative variance
        cumvar = np.cumsum(ev)
        axes[1].plot(range(1, len(cumvar) + 1), cumvar, marker='o', color='coral', linewidth=2)
        axes[1].axhline(y=0.95, color='r', linestyle='--', label='95%')
        axes[1].axhline(y=0.90, color='orange', linestyle='--', label='90%')
        axes[1].set_xlabel('Components')
        axes[1].set_ylabel('Cumulative Variance')
        axes[1].set_title('Cumulative Variance')
        axes[1].legend()
        axes[1].grid(alpha=0.3)

        # 3. 2D scatter (first 2 PCA components from training data)
        # Re-extract numerical, apply PCA for plotting
        X_num = self.X_train[:, :self.pca.n_components_] if self.feature_mode in ['C', 'D'] else None
        if X_num is not None and X_num.shape[1] >= 2:
            scatter = axes[2].scatter(X_num[:, 0], X_num[:, 1], c=self.y_train,
                                      cmap='viridis', alpha=0.6, edgecolors='k', linewidth=0.5)
            axes[2].set_xlabel(f'PC1 ({ev[0]*100:.1f}%)')
            axes[2].set_ylabel(f'PC2 ({ev[1]*100:.1f}%)')
            axes[2].set_title('PCA 2D Projection (Numerical)')
            plt.colorbar(scatter, ax=axes[2], label='Target')
        else:
            axes[2].text(0.5, 0.5, 'Need ≥ 2 PCA components', ha='center', va='center')
            axes[2].axis('off')

        plt.tight_layout()
        plt.show()

    # =========================================================================
    # STEP 3: TRAIN, COMPARE, OPTIMIZE, SELECT
    # =========================================================================

    def train_all_models(self, cv=5):
        """Train all models with cross-validation."""
        print("\n" + "=" * 70)
        print("TRAINING ALL MODELS")
        print("=" * 70)

        scoring = 'accuracy' if self.task == 'classification' else 'r2'

        for name, model in self.models.items():
            print(f"\n  ⚙️  {name}...")
            model.fit(self.X_train, self.y_train)

            cv_scores = cross_val_score(model, self.X_train, self.y_train, cv=cv, scoring=scoring)
            y_pred = model.predict(self.X_test)

            if self.task == 'classification':
                metrics = {
                    'accuracy': accuracy_score(self.y_test, y_pred),
                    'precision': precision_score(self.y_test, y_pred, average='weighted', zero_division=0),
                    'recall': recall_score(self.y_test, y_pred, average='weighted', zero_division=0),
                    'f1': f1_score(self.y_test, y_pred, average='weighted', zero_division=0),
                    'cv_score': cv_scores.mean(),
                    'cv_std': cv_scores.std()
                }
            else:
                metrics = {
                    'r2': r2_score(self.y_test, y_pred),
                    'rmse': np.sqrt(mean_squared_error(self.y_test, y_pred)),
                    'mae': mean_absolute_error(self.y_test, y_pred),
                    'cv_score': cv_scores.mean(),
                    'cv_std': cv_scores.std()
                }

            self.results[name] = {'model': model, 'metrics': metrics, 'predictions': y_pred}
            print(f"    CV: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

        print("\n✓ All models trained")

    def compare_models(self):
        """Compare all trained models."""
        print("\n" + "=" * 70)
        print("MODEL COMPARISON")
        print("=" * 70)

        rows = []
        for name, res in self.results.items():
            if 'metrics' in res:
                row = {'Model': name}
                row.update(res['metrics'])
                rows.append(row)

        df = pd.DataFrame(rows)
        sort_col = 'accuracy' if self.task == 'classification' else 'r2'
        df = df.sort_values(sort_col, ascending=False)
        print(df.to_string(index=False))
        return df

    def optimize_model(self, model_name, param_grid, search_type='grid', cv=5, n_iter=50):
        """Optimize hyperparameters for a specific model."""
        print("\n" + "=" * 70)
        print(f"OPTIMIZING {model_name}")
        print("=" * 70)

        model = self.models[model_name]
        scoring = 'accuracy' if self.task == 'classification' else 'r2'

        if search_type == 'grid':
            search = GridSearchCV(model, param_grid, cv=cv, scoring=scoring, n_jobs=-1, verbose=1)
        else:
            search = RandomizedSearchCV(model, param_grid, n_iter=n_iter, cv=cv,
                                        scoring=scoring, n_jobs=-1, verbose=1, random_state=self.random_state)

        search.fit(self.X_train, self.y_train)
        y_pred = search.best_estimator_.predict(self.X_test)
        test_score = accuracy_score(self.y_test, y_pred) if self.task == 'classification' else r2_score(self.y_test, y_pred)

        print(f"\n✓ Best params: {search.best_params_}")
        print(f"✓ Best CV score: {search.best_score_:.4f}")
        print(f"✓ Test score: {test_score:.4f}")

        self.results[f"{model_name} (Optimized)"] = {
            'model': search.best_estimator_,
            'metrics': {'cv_score': search.best_score_, 'cv_std': 0},
            'test_score': test_score
        }
        # Update accuracy/r2 in metrics for comparison
        metric_key = 'accuracy' if self.task == 'classification' else 'r2'
        self.results[f"{model_name} (Optimized)"]['metrics'][metric_key] = test_score

        return search.best_estimator_, search.best_params_

    def select_best_model(self):
        """Select best performing model."""
        print("\n" + "=" * 70)
        print("SELECTING BEST MODEL")
        print("=" * 70)

        best_name, best_score = None, -np.inf
        metric_key = 'accuracy' if self.task == 'classification' else 'r2'

        for name, res in self.results.items():
            score = res.get('metrics', {}).get(metric_key, res.get('test_score', -np.inf))
            if score > best_score:
                best_score = score
                best_name = name

        if best_name is None:
            print("✗ No models found. Train models first.")
            return None, None

        self.best_model = self.results[best_name]['model']
        print(f"✓ Best Model: {best_name}")
        print(f"✓ Score: {best_score:.4f}")
        return self.best_model, best_name

    # =========================================================================
    # STEP 4: PREDICT — mirrors exact training pipeline
    # =========================================================================

    def _encode_new_data(self, X_new):
        """Encode categorical features in new data using saved encoders."""
        for col in self.categorical_features:
            if col not in X_new.columns or col not in self.categorical_encoders:
                continue
            enc = self.categorical_encoders[col]
            try:
                if isinstance(enc, OneHotEncoder):
                    encoded = enc.transform(X_new[[col]])
                    new_cols = [f"{col}_{cat}" for cat in enc.categories_[0]]
                    enc_df = pd.DataFrame(encoded, columns=new_cols, index=X_new.index)
                    X_new = pd.concat([X_new.drop(col, axis=1), enc_df], axis=1)
                elif isinstance(enc, LabelEncoder):
                    known = set(enc.classes_)
                    X_new[col] = X_new[col].astype(str).apply(lambda x: x if x in known else enc.classes_[0])
                    X_new[col] = enc.transform(X_new[col])
            except Exception as e:
                print(f"  ⚠️  Encoding error for {col}: {e}")
        return X_new

    def predict(self, X_new, return_proba=False):
        """
        Make predictions — applies the exact same pipeline as training:
            Encode → Split (num | cat) → PCA/Chi2 → Combine → Scale → Predict

        Args:
            X_new: New data (DataFrame or numpy array)
            return_proba: Return class probabilities (classification only)
        """
        if self.best_model is None:
            print("✗ No best model. Run select_best_model() first.")
            return None

        print("\n" + "=" * 70)
        print(f"PREDICTING — Mode {self.feature_mode}")
        print("=" * 70)

        # Convert to DataFrame
        if isinstance(X_new, np.ndarray):
            cols = self.feature_names if len(self.feature_names) == X_new.shape[1] else [f'feature_{i}' for i in range(X_new.shape[1])]
            X_new = pd.DataFrame(X_new, columns=cols)
        elif not isinstance(X_new, pd.DataFrame):
            X_new = pd.DataFrame(X_new)

        print(f"✓ Input shape: {X_new.shape}")

        # Step 1: Encode categorical
        if len(self.categorical_features) > 0:
            print("✓ Step 1: Encoding categorical features...")
            X_new = self._encode_new_data(X_new)
            print(f"  Shape after encoding: {X_new.shape}")

        # Convert to numpy
        X_array = X_new.values.astype(float)

        # Step 2: Split into numerical and categorical using saved indices
        X_num = X_array[:, self.num_indices]
        X_cat = X_array[:, self.cat_indices] if len(self.cat_indices) > 0 else None
        print(f"✓ Step 2: Split — Num: {X_num.shape[1]} | Cat: {X_cat.shape[1] if X_cat is not None else 0}")

        # Step 3: Apply PCA on numerical (if mode C or D)
        if self.feature_mode in ['C', 'D'] and self.pca is not None:
            print("✓ Step 3: Applying PCA on numerical features...")
            X_num = self.pca.transform(X_num)
            print(f"  Numerical → {X_num.shape[1]} PCA components")
        else:
            print(f"✓ Step 3: Keeping all {X_num.shape[1]} numerical features")

        # Step 4: Apply Chi2 on categorical (if mode B or D)
        if self.feature_mode in ['B', 'D'] and self.chi2_selector is not None and X_cat is not None:
            print("✓ Step 4: Applying Chi2 on categorical features...")
            X_cat_nn = X_cat - self.chi2_min_offset + 1e-10
            X_cat = self.chi2_selector.transform(X_cat_nn)
            print(f"  Categorical → {X_cat.shape[1]} selected features")
        elif X_cat is not None:
            print(f"✓ Step 4: Keeping all {X_cat.shape[1]} categorical features")

        # Step 5: Combine
        if X_cat is not None:
            X_combined = np.hstack([X_num, X_cat])
        else:
            X_combined = X_num
        print(f"✓ Step 5: Combined shape: {X_combined.shape}")

        # Step 6: Scale
        print("✓ Step 6: Scaling...")
        X_scaled = self.scaler.transform(X_combined)

        print(f"✓ Final shape for prediction: {X_scaled.shape}")

        # Step 7: Predict
        try:
            predictions = self.best_model.predict(X_scaled)
            print(f"✓ Generated {len(predictions)} predictions")
        except Exception as e:
            print(f"✗ Prediction error: {e}")
            return None

        # Probabilities
        if return_proba and self.task == 'classification':
            if hasattr(self.best_model, 'predict_proba'):
                proba = self.best_model.predict_proba(X_scaled)
                print(f"✓ Probability shape: {proba.shape}")
                return predictions, proba
            else:
                print("⚠️  Model does not support predict_proba")

        return predictions

    # =========================================================================
    # STEP 5: EVALUATE & VISUALIZE
    # =========================================================================

    def evaluate_predictions(self, y_true, y_pred):
        """Evaluate predictions against true values."""
        print("\n" + "=" * 70)
        print("PREDICTION EVALUATION")
        print("=" * 70)

        if self.task == 'classification':
            metrics = {
                'accuracy': accuracy_score(y_true, y_pred),
                'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
                'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
                'f1': f1_score(y_true, y_pred, average='weighted', zero_division=0)
            }
            print(f"  Accuracy:  {metrics['accuracy']:.4f}")
            print(f"  Precision: {metrics['precision']:.4f}")
            print(f"  Recall:    {metrics['recall']:.4f}")
            print(f"  F1 Score:  {metrics['f1']:.4f}")
            print(f"\n  Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}")
        else:
            metrics = {
                'r2': r2_score(y_true, y_pred),
                'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
                'mae': mean_absolute_error(y_true, y_pred)
            }
            print(f"  R² Score: {metrics['r2']:.4f}")
            print(f"  RMSE:     {metrics['rmse']:.4f}")
            print(f"  MAE:      {metrics['mae']:.4f}")
        return metrics

    def plot_results(self, figsize=(15, 10)):
        """Plot model comparison and best model results."""
        if not self.results:
            print("✗ No results. Train models first.")
            return

        fig, axes = plt.subplots(2, 2, figsize=figsize)
        metric_key = 'accuracy' if self.task == 'classification' else 'r2'
        metric_label = 'Accuracy' if self.task == 'classification' else 'R² Score'

        # Collect only models with metrics
        names, scores, cv_scores, cv_stds = [], [], [], []
        for name, res in self.results.items():
            if 'metrics' in res and metric_key in res['metrics']:
                names.append(name)
                scores.append(res['metrics'][metric_key])
                cv_scores.append(res['metrics'].get('cv_score', 0))
                cv_stds.append(res['metrics'].get('cv_std', 0))

        # 1. Model comparison
        if names:
            axes[0, 0].barh(names, scores, color='steelblue')
            axes[0, 0].set_xlabel(metric_label)
            axes[0, 0].set_title(f'Model Comparison — {metric_label}')
            axes[0, 0].grid(axis='x', alpha=0.3)
        else:
            axes[0, 0].text(0.5, 0.5, 'No scores', ha='center')
            axes[0, 0].axis('off')

        # 2. CV scores
        if names:
            axes[0, 1].barh(names, cv_scores, xerr=cv_stds, color='coral', alpha=0.7)
            axes[0, 1].set_xlabel('CV Score')
            axes[0, 1].set_title('Cross-Validation Scores')
            axes[0, 1].grid(axis='x', alpha=0.3)
        else:
            axes[0, 1].text(0.5, 0.5, 'No CV scores', ha='center')
            axes[0, 1].axis('off')

        # 3. Confusion matrix / Residual plot
        best_model, best_name = self.select_best_model()
        if best_model is not None:
            try:
                y_pred = best_model.predict(self.X_test)
                if self.task == 'classification':
                    cm = confusion_matrix(self.y_test, y_pred)
                    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1, 0])
                    axes[1, 0].set_title(f'Confusion Matrix — {best_name}')
                    axes[1, 0].set_ylabel('True')
                    axes[1, 0].set_xlabel('Predicted')
                else:
                    residuals = self.y_test - y_pred
                    axes[1, 0].scatter(y_pred, residuals, alpha=0.6)
                    axes[1, 0].axhline(0, color='r', linestyle='--')
                    axes[1, 0].set_xlabel('Predicted')
                    axes[1, 0].set_ylabel('Residuals')
                    axes[1, 0].set_title(f'Residual Plot — {best_name}')
            except Exception as e:
                axes[1, 0].text(0.5, 0.5, f'Plot error: {str(e)[:40]}', ha='center')
                axes[1, 0].axis('off')
        else:
            axes[1, 0].text(0.5, 0.5, 'No best model', ha='center')
            axes[1, 0].axis('off')

        # 4. Feature importance / Actual vs Predicted
        if best_model is not None:
            try:
                y_pred = best_model.predict(self.X_test)
                if self.task == 'classification':
                    if hasattr(best_model, 'feature_importances_'):
                        imp = best_model.feature_importances_
                        n = min(10, len(imp))
                        idx = np.argsort(imp)[::-1][:n]
                        axes[1, 1].barh(range(n), imp[idx], color='green', alpha=0.7)
                        axes[1, 1].set_yticks(range(n))
                        axes[1, 1].set_yticklabels([f'Feature {i}' for i in idx])
                        axes[1, 1].set_xlabel('Importance')
                        axes[1, 1].set_title(f'Feature Importance — {best_name}')
                        axes[1, 1].invert_yaxis()
                    elif hasattr(best_model, 'coef_'):
                        coef = np.abs(best_model.coef_[0] if len(best_model.coef_.shape) > 1 else best_model.coef_)
                        n = min(10, len(coef))
                        idx = np.argsort(coef)[::-1][:n]
                        axes[1, 1].barh(range(n), coef[idx], color='green', alpha=0.7)
                        axes[1, 1].set_yticks(range(n))
                        axes[1, 1].set_yticklabels([f'Feature {i}' for i in idx])
                        axes[1, 1].set_xlabel('|Coefficient|')
                        axes[1, 1].set_title(f'Coefficients — {best_name}')
                        axes[1, 1].invert_yaxis()
                    else:
                        axes[1, 1].text(0.5, 0.5, 'Feature importance\nnot available', ha='center')
                        axes[1, 1].axis('off')
                else:
                    axes[1, 1].scatter(self.y_test, y_pred, alpha=0.6, edgecolors='k', linewidth=0.5)
                    lo = min(self.y_test.min(), y_pred.min())
                    hi = max(self.y_test.max(), y_pred.max())
                    axes[1, 1].plot([lo, hi], [lo, hi], 'r--', lw=2, label='Perfect')
                    axes[1, 1].set_xlabel('Actual')
                    axes[1, 1].set_ylabel('Predicted')
                    axes[1, 1].set_title(f'Actual vs Predicted — {best_name}')
                    axes[1, 1].legend()
            except Exception as e:
                axes[1, 1].text(0.5, 0.5, f'Plot error: {str(e)[:40]}', ha='center')
                axes[1, 1].axis('off')
        else:
            axes[1, 1].text(0.5, 0.5, 'No best model', ha='center')
            axes[1, 1].axis('off')

        plt.tight_layout()
        plt.show()

    # =========================================================================
    # SAVE / LOAD
    # =========================================================================

    def save_model(self, filepath='best_model.pkl'):
        """Save model and entire preprocessing pipeline."""
        import pickle
        if self.best_model is None:
            print("✗ No model to save.")
            return
        package = {
            'model': self.best_model,
            'scaler': self.scaler,
            'pca': self.pca,
            'chi2_selector': self.chi2_selector,
            'chi2_min_offset': getattr(self, 'chi2_min_offset', None),
            'categorical_encoders': self.categorical_encoders,
            'categorical_features': self.categorical_features,
            'numerical_features': self.numerical_features,
            'feature_names': self.feature_names,
            'num_indices': self.num_indices,
            'cat_indices': self.cat_indices,
            'feature_mode': self.feature_mode,
            'task': self.task
        }

        with open(filepath, 'wb') as f:
            pickle.dump(package, f)
        print(f"✓ Model saved: {filepath}")

    def load_model(self, filepath='best_model.pkl'):
        """Load model and preprocessing pipeline."""
        import pickle
        with open(filepath, 'rb') as f:
            pkg = pickle.load(f)

        self.best_model = pkg['model']
        self.scaler = pkg['scaler']
        self.pca = pkg['pca']
        self.chi2_selector = pkg['chi2_selector']
        self.chi2_min_offset = pkg.get('chi2_min_offset')
        self.categorical_encoders = pkg['categorical_encoders']
        self.categorical_features = pkg['categorical_features']
        self.numerical_features = pkg['numerical_features']
        self.feature_names = pkg['feature_names']
        self.num_indices = pkg['num_indices']
        self.cat_indices = pkg['cat_indices']
        self.feature_mode = pkg['feature_mode']
        self.task = pkg['task']

        print(f"✓ Model loaded: {filepath}")
        print(f"✓ Feature mode: {self.feature_mode} | Task: {self.task}")